# Trafo Tests

In [1]:
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.insert(0,'/home/mmo-kra/git/dpsim/build')
import dpsimpy


# %matplotlib widget

epsilon = 1e-12
epsilon = 0.068

PEAK1PH_TO_RMS3PH = np.sqrt(3/2)

## SP Trafo Elements Simulation

In [2]:
time_step = 0.00005
final_time = 1
sim_name = 'SP_Trafo_Elements'
frequency = 50
omega = 2 * np.pi * frequency
dpsimpy.Logger.set_log_dir('logs/' + sim_name)

voltage_hv_side = 100000
voltage_mv_side = 10000
trafo_resistance = 1
trafo_inductance = 0.1
trafo_power = 1e6
load_resistance_hv_side = 10000
ratio = voltage_hv_side / voltage_mv_side

n1 = dpsimpy.sp.SimNode('n1')
n1.set_initial_voltage(complex(voltage_hv_side, 0))
n2 = dpsimpy.sp.SimNode('n2')
n2.set_initial_voltage(complex(99989.01435, -314.09334))
vn1 = dpsimpy.sp.SimNode('vn1')
vn1.set_initial_voltage(complex(99990.001099, 0.031409))
gnd = dpsimpy.sp.SimNode.gnd

vs = dpsimpy.sp.ph1.VoltageSource('v_1')
trafo_res = dpsimpy.sp.ph1.Resistor('trafo_res')
trafo_ind = dpsimpy.sp.ph1.Inductor('trafo_ind')
load_res = dpsimpy.sp.ph1.Resistor('r_1')

vs.connect([gnd, n1])
trafo_res.connect([n1, vn1])
trafo_ind.connect([vn1, n2])
load_res.connect([n2, gnd])

vs.set_parameters(complex(voltage_hv_side, 0))
trafo_res.set_parameters(trafo_resistance)
trafo_ind.set_parameters(trafo_inductance)
load_res.set_parameters(load_resistance_hv_side)

sys = dpsimpy.SystemTopology(50, [n1, n2, vn1], [vs, trafo_res, trafo_ind, load_res])

logger = dpsimpy.Logger(sim_name)
logger.log_attribute('v1', 'v', n1)
logger.log_attribute('v2', 'v', n2)
logger.log_attribute('vn1', 'v', vn1)
logger.log_attribute('itrafo', 'i_intf', trafo_ind)

sim = dpsimpy.Simulation(sim_name)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.add_logger(logger)
sim.set_domain(dpsimpy.Domain.SP)

sim.run()

[11:57:08.959480 MnaSolverFactory info] creating SparseLUAdapter solver implementation


## SP Trafo Component Simulation

In [3]:
time_step = 0.00005
final_time = 1
sim_name = 'SP_Trafo_Component'
dpsimpy.Logger.set_log_dir('logs/' + sim_name)

voltage_hv_side = 100000
voltage_mv_side = 10000
trafo_resistance = 1
trafo_inductance = 0.1
load_resistance_hv_side = 10000
ratio = voltage_hv_side / voltage_mv_side
load_resistance_mv_side = load_resistance_hv_side / ratio**2

n1 = dpsimpy.sp.SimNode('n1')
n1.set_initial_voltage(complex(voltage_hv_side, 0))
n2 = dpsimpy.sp.SimNode('n2')
n2.set_initial_voltage(complex(9998.901435, -31.409334))
gnd = dpsimpy.sp.SimNode.gnd

vs = dpsimpy.sp.ph1.VoltageSource('v_1', dpsimpy.LogLevel.debug)
trafo = dpsimpy.sp.ph1.Transformer('trafo', 'trafo', dpsimpy.LogLevel.debug)
load_res = dpsimpy.sp.ph1.Resistor('r_1', dpsimpy.LogLevel.debug)

vs.connect([gnd, n1])
trafo.connect([n1, n2])
load_res.connect([n2, gnd])

vs.set_parameters(complex(voltage_hv_side, 0))
trafo.set_parameters(voltage_hv_side, voltage_mv_side, trafo_power, ratio, 0,
    trafo_resistance, trafo_inductance)
load_res.set_parameters(load_resistance_mv_side)

sys = dpsimpy.SystemTopology(50, [n1, n2], [vs, trafo, load_res])

logger = dpsimpy.Logger(sim_name)
logger.log_attribute('v1', 'v', n1)
logger.log_attribute('v2', 'v', n2)
logger.log_attribute('itrafo', 'primary_current', trafo) #primary side current
#logger.log_attribute('itrafo_sec', 'secondary_current', trafo)
logger.log_attribute('LV_trafo_prim', 'primary_voltage_LVside', trafo)
logger.log_attribute('LV_trafo_sec', 'secondary_voltage_LVside', trafo)


sim = dpsimpy.Simulation(sim_name)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.add_logger(logger)
sim.set_domain(dpsimpy.Domain.SP)

sim.run()

[11:57:10.933922 MnaSolverFactory info] creating SparseLUAdapter solver implementation


## DP Trafo Elements Simulation

In [4]:
time_step = 0.00005
final_time = 1
sim_name = 'DP_Trafo_Elements'
frequency = 50
omega = 2 * np.pi * frequency
dpsimpy.Logger.set_log_dir('logs/' + sim_name)

voltage_hv_side = 100000
voltage_mv_side = 10000
trafo_resistance = 1
trafo_inductance = 0.1
load_resistance_hv_side = 10000
ratio = voltage_hv_side / voltage_mv_side

n1 = dpsimpy.dp.SimNode('n1')
n1.set_initial_voltage(complex(voltage_hv_side, 0))
n2 = dpsimpy.dp.SimNode('n2')
n2.set_initial_voltage(complex(99989.01435, -314.09334))
vn1 = dpsimpy.dp.SimNode('vn1')
vn1.set_initial_voltage(complex(99990.001099, 0.031409))
gnd = dpsimpy.dp.SimNode.gnd

vs = dpsimpy.dp.ph1.VoltageSource('v_1', dpsimpy.LogLevel.debug)
trafo_res = dpsimpy.dp.ph1.Resistor('trafo_res', dpsimpy.LogLevel.debug)
trafo_ind = dpsimpy.dp.ph1.Inductor('trafo_ind', dpsimpy.LogLevel.debug)
load_res = dpsimpy.dp.ph1.Resistor('r_1', dpsimpy.LogLevel.debug)

vs.connect([gnd, n1])
trafo_res.connect([n1, vn1])
trafo_ind.connect([vn1, n2])
load_res.connect([n2, gnd])

vs.set_parameters(complex(voltage_hv_side, 0))
trafo_res.set_parameters(trafo_resistance)
trafo_ind.set_parameters(trafo_inductance)
load_res.set_parameters(load_resistance_hv_side)

sys = dpsimpy.SystemTopology(50, [n1, n2, vn1], [vs, trafo_res, trafo_ind, load_res])

logger = dpsimpy.Logger(sim_name)
logger.log_attribute('v1', 'v', n1)
logger.log_attribute('v2', 'v', n2)
logger.log_attribute('vn1', 'v', vn1)
logger.log_attribute('itrafo', 'i_intf', trafo_ind)

sim = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.debug)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.add_logger(logger)
sim.set_domain(dpsimpy.Domain.DP)

sim.run()

[11:57:12.523448 DP_Trafo_Elements info] Initialize simulation: DP_Trafo_Elements
[11:57:12.523628 MnaSolverFactory info] creating SparseLUAdapter solver implementation
[11:57:12.525063 DP_Trafo_Elements info] Scheduling tasks.
[11:57:12.525426 DP_Trafo_Elements info] Scheduling done.
[11:57:12.525429 DP_Trafo_Elements info] Opening interfaces.
[11:57:12.525431 DP_Trafo_Elements info] Start synchronization with remotes on interfaces
[11:57:12.525432 DP_Trafo_Elements info] Synchronized simulation start with remotes
[11:57:12.525435 DP_Trafo_Elements info] Start simulation: DP_Trafo_Elements
[11:57:12.525438 DP_Trafo_Elements info] Time step: 5.000000e-05
[11:57:12.525441 DP_Trafo_Elements info] Final time: 1.000000e+00
[11:57:14.469084 DP_Trafo_Elements info] Simulation calculation time: 1.943595
[11:57:14.470936 DP_Trafo_Elements info] Simulation finished.


## DP Trafo Component Simulation

In [5]:
time_step = 0.00005
final_time = 1
sim_name = 'DP_Trafo_Component'
dpsimpy.Logger.set_log_dir('logs/' + sim_name)

voltage_hv_side = 100000
voltage_mv_side = 10000
trafo_resistance = 1
trafo_inductance = 0.1
load_resistance_hv_side = 10000
ratio = voltage_hv_side / voltage_mv_side
load_resistance_mv_side = load_resistance_hv_side / ratio**2

n1 = dpsimpy.dp.SimNode('n1')
n1.set_initial_voltage(complex(voltage_hv_side, 0))
n2 = dpsimpy.dp.SimNode('n2')
n2.set_initial_voltage(complex(9998.901435, -31.409334))
gnd = dpsimpy.dp.SimNode.gnd

vs = dpsimpy.dp.ph1.VoltageSource('v_1', dpsimpy.LogLevel.debug)
trafo = dpsimpy.dp.ph1.Transformer('trafo', dpsimpy.LogLevel.debug)
load_res = dpsimpy.dp.ph1.Resistor('r_1', dpsimpy.LogLevel.debug)

vs.connect([gnd, n1])
trafo.connect([n1, n2])
load_res.connect([n2, gnd])

vs.set_parameters(complex(voltage_hv_side, 0))
trafo.set_parameters(voltage_hv_side, voltage_mv_side, ratio, 0, trafo_resistance, trafo_inductance)
load_res.set_parameters(load_resistance_mv_side)

sys = dpsimpy.SystemTopology(50, [n1, n2], [vs, trafo, load_res])

logger = dpsimpy.Logger(sim_name)
logger.log_attribute('v1', 'v', n1)
logger.log_attribute('v2', 'v', n2)
logger.log_attribute('itrafo', 'i_intf', trafo)
logger.log_attribute('vTrafo_p', 'primary_voltage_LVside', trafo)
logger.log_attribute('vTrafo_s', 'secondary_voltage_LVside', trafo)

sim = dpsimpy.Simulation(sim_name)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.add_logger(logger)
sim.set_domain(dpsimpy.Domain.DP)

sim.run()

[11:57:14.550953 MnaSolverFactory info] creating SparseLUAdapter solver implementation
[*** LOG ERROR #0001 ***] [2023-06-07 11:57:14] [trafo] {argument not found}


## EMT Trafo Elements Simulation

In [6]:
time_step = 0.00005
final_time = 1
sim_name = 'EMT_Trafo_Elements'
frequency = 50
omega = 2 * np.pi * frequency
dpsimpy.Logger.set_log_dir('logs/' + sim_name)

voltage_hv_side = 100000
voltage_mv_side = 10000
trafo_resistance = 1
trafo_inductance = 0.1
trafo_power = 1e6
p_snub = dpsimpy.P_SNUB_TRANSFORMER * trafo_power
q_snub = dpsimpy.Q_SNUB_TRANSFORMER * trafo_power
load_resistance_hv_side = 10000
ratio = voltage_hv_side / voltage_mv_side
snubber_resistance_hv_side = np.abs(voltage_hv_side)**2 / p_snub
snubber_resistance_mv_side_to_hv_side = ratio**2 * np.abs(voltage_mv_side)**2 / p_snub
snubber_capacitance_mv_side_to_hv_side = 1/(omega * ratio**2 * np.abs(voltage_mv_side)**2 / q_snub) 


n1 = dpsimpy.emt.SimNode('n1', dpsimpy.PhaseType.ABC)
n1.set_initial_voltage(dpsimpy.Math.single_phase_variable_to_three_phase(complex(voltage_hv_side, 0)))
n2 = dpsimpy.emt.SimNode('n2', dpsimpy.PhaseType.ABC)
n2.set_initial_voltage(dpsimpy.Math.single_phase_variable_to_three_phase(complex(99989.159386, -314.413354)))
vn1 = dpsimpy.emt.SimNode('vn1', dpsimpy.PhaseType.ABC)
vn1.set_initial_voltage(dpsimpy.Math.single_phase_variable_to_three_phase(complex(99989.991069, 0.026473)))
gnd = dpsimpy.emt.SimNode.gnd

vs = dpsimpy.emt.ph3.VoltageSource('v_1')
trafo_res = dpsimpy.emt.ph3.Resistor('trafo_res')
trafo_snubber_res_hv_side = dpsimpy.emt.ph3.Resistor('trafo_snub_res_mv') ##FIXME: Is the naming correct here?
trafo_snubber_res_mv_side = dpsimpy.emt.ph3.Resistor('trafo_snub_res_hv')
trafo_snubber_cap_mv_side = dpsimpy.emt.ph3.Capacitor('trafo_snub_cap_mv')
trafo_ind = dpsimpy.emt.ph3.Inductor('trafo_ind')
load_res = dpsimpy.emt.ph3.Resistor('r_1')

vs.connect([gnd, n1])
trafo_res.connect([n1, vn1])
trafo_ind.connect([vn1, n2])
trafo_snubber_res_hv_side.connect([n1, gnd])
trafo_snubber_res_mv_side.connect([n2, gnd])
trafo_snubber_cap_mv_side.connect([n2, gnd])
load_res.connect([n2, gnd])

vs.set_parameters(dpsimpy.Math.single_phase_variable_to_three_phase(complex(voltage_hv_side, 0)), 50)
trafo_res.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(trafo_resistance))
trafo_ind.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(trafo_inductance))
trafo_snubber_res_hv_side.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(snubber_resistance_hv_side))
trafo_snubber_res_mv_side.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(snubber_resistance_mv_side_to_hv_side))
trafo_snubber_cap_mv_side.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(snubber_capacitance_mv_side_to_hv_side))
load_res.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(load_resistance_hv_side))

sys = dpsimpy.SystemTopology(50, [n1, n2, vn1], [vs, trafo_res, trafo_ind, trafo_snubber_res_hv_side, trafo_snubber_res_mv_side, trafo_snubber_cap_mv_side, load_res])

logger = dpsimpy.Logger(sim_name)
logger.log_attribute('v1', 'v', n1)
logger.log_attribute('v2', 'v', n2)
logger.log_attribute('itrafo', 'i_intf', trafo_ind)

sim = dpsimpy.Simulation(sim_name)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.add_logger(logger)
sim.set_domain(dpsimpy.Domain.EMT)

sim.run()

[11:57:16.231398 MnaSolverFactory info] creating SparseLUAdapter solver implementation


## EMT Trafo Component Simulation

In [ ]:
time_step = 0.00005
final_time = 1
sim_name = 'EMT_Trafo_Component'
dpsimpy.Logger.set_log_dir('logs/' + sim_name)

voltage_hv_side = 100000
voltage_mv_side = 10000
trafo_resistance = 1
trafo_inductance = 0.1
load_resistance_hv_side = 10000
ratio = voltage_hv_side / voltage_mv_side
load_resistance_mv_side = load_resistance_hv_side / ratio**2

n1 = dpsimpy.emt.SimNode('n1', dpsimpy.PhaseType.ABC)
n1.set_initial_voltage(dpsimpy.Math.single_phase_variable_to_three_phase(complex(voltage_hv_side, 0)))
n2 = dpsimpy.emt.SimNode('n2', dpsimpy.PhaseType.ABC)
n2.set_initial_voltage(dpsimpy.Math.single_phase_variable_to_three_phase(complex(9998.9159386, -31.4413354)))
gnd = dpsimpy.emt.SimNode.gnd

vs = dpsimpy.emt.ph3.VoltageSource('v_1', dpsimpy.LogLevel.debug)
trafo = dpsimpy.emt.ph3.Transformer('trafo', 'trafo', dpsimpy.LogLevel.debug)
load_res = dpsimpy.emt.ph3.Resistor('r_1', dpsimpy.LogLevel.debug)

vs.connect([gnd, n1])
trafo.connect([n1, n2])
load_res.connect([n2, gnd])

vs.set_parameters(dpsimpy.Math.single_phase_variable_to_three_phase(complex(voltage_hv_side, 0)), 50)
trafo.set_parameters(voltage_hv_side, voltage_mv_side, ratio, 0,
    dpsimpy.Math.single_phase_parameter_to_three_phase(trafo_resistance),
    dpsimpy.Math.single_phase_parameter_to_three_phase(trafo_inductance))
load_res.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(load_resistance_mv_side))

sys = dpsimpy.SystemTopology(50, [n1, n2], [vs, trafo, load_res])

logger = dpsimpy.Logger(sim_name)
logger.log_attribute('v1', 'v', n1)
logger.log_attribute('v2', 'v', n2)
logger.log_attribute('itrafo', 'i_intf', trafo)

sim = dpsimpy.Simulation(sim_name)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.add_logger(logger)
sim.set_domain(dpsimpy.Domain.EMT)

sim.run()

: 

: 

## SP Trafo with elements

In [ ]:
work_dir = 'logs/SP_Trafo_Elements/'
log_name = 'SP_Trafo_Elements'
print(work_dir + log_name + '.csv')
trafo_elements = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')
trafo_elements_sp_shifted = ts.frequency_shift_list(trafo_elements, 50)

In [ ]:
plt.figure()
plt.plot(trafo_elements_sp_shifted['v1_shift'].time, trafo_elements_sp_shifted['v1_shift'].values, label='v1_shift')
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(trafo_elements_sp_shifted['itrafo_shift'].time, trafo_elements_sp_shifted['itrafo_shift'].values, label='itrafo_shift')
plt.legend()
plt.show()

## SP Trafo composite model

In [ ]:
work_dir = 'logs/SP_Trafo_Component/'
log_name = 'SP_Trafo_Component'
print(work_dir + log_name + '.csv')
trafo_component = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')
trafo_component_sp_shifted = ts.frequency_shift_list(trafo_component, 50)
trafo_component_sp_shifted['itrafo_shift'].values = -trafo_component_sp_shifted['itrafo_shift'].values

In [ ]:
plt.figure()
plt.plot(trafo_component_sp_shifted['v1_shift'].time, trafo_component_sp_shifted['v1_shift'].values, label='v1_shift')
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(trafo_component_sp_shifted['itrafo_shift'].time, trafo_component_sp_shifted['itrafo_shift'].values, label='itrafo_shift')
plt.legend()
plt.show()

## Error for SP Trafo

In [ ]:
plt.figure()
for name in ['v1_shift', 'itrafo_shift']:
    plt.plot(trafo_elements_sp_shifted[name].time, trafo_elements_sp_shifted[name].values - trafo_component_sp_shifted[name].values, label=name+'_error')
plt.legend()
plt.show()

## Assertion for SP Trafo

In [ ]:
errors_sp_shifted = []
for name in ['v1_shift', 'itrafo_shift']:
    errors_sp_shifted.append(np.absolute(trafo_elements_sp_shifted[name].values - trafo_component_sp_shifted[name].values).max())
    print(name + ': ' + str(errors_sp_shifted[-1]))
assert np.max(errors_sp_shifted) < epsilon

## DP Trafo with elements

In [ ]:
work_dir = 'logs/DP_Trafo_Elements/'
log_name = 'DP_Trafo_Elements'
print(work_dir + log_name + '.csv')
trafo_elements = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')
trafo_elements_dp_shifted = ts.frequency_shift_list(trafo_elements, 50)

In [ ]:
plt.figure()
plt.plot(trafo_elements_dp_shifted['v1_shift'].time, trafo_elements_dp_shifted['v1_shift'].values, label='v1_shift')
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(trafo_elements_dp_shifted['itrafo_shift'].time, trafo_elements_dp_shifted['itrafo_shift'].values, label='itrafo_shift')
plt.legend()
plt.show()

## DP Trafo composite model

In [ ]:
work_dir = 'logs/DP_Trafo_Component/'
log_name = 'DP_Trafo_Component'
print(work_dir + log_name + '.csv')
trafo_component = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')
trafo_component_dp_shifted = ts.frequency_shift_list(trafo_component, 50)
trafo_component_dp_shifted['itrafo_shift'].values = -trafo_component_dp_shifted['itrafo_shift'].values

In [ ]:
plt.figure()
plt.plot(trafo_component_dp_shifted['v1_shift'].time, trafo_component_dp_shifted['v1_shift'].values, label='v1_shift')
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(trafo_component_dp_shifted['itrafo_shift'].time, trafo_component_dp_shifted['itrafo_shift'].values, label='itrafo_shift')
plt.legend()
plt.show()

## Error for DP Trafo

In [ ]:
plt.figure()
for name in ['v1_shift', 'itrafo_shift']:
    plt.plot(trafo_elements_dp_shifted[name].time, trafo_elements_dp_shifted[name].values - trafo_component_dp_shifted[name].values, label=name+'_error')
plt.legend()
plt.show()

## Assertion for DP Trafo

In [ ]:
errors_dp_shifted = []
for name in ['v1_shift', 'itrafo_shift']:
    errors_dp_shifted.append(np.absolute(trafo_elements_dp_shifted[name].values - trafo_component_dp_shifted[name].values).max())
    print(name + ': ' + str(errors_dp_shifted[-1]))
assert np.max(errors_dp_shifted) < epsilon

## EMT Trafo with elements

In [ ]:
work_dir = 'logs/EMT_Trafo_Elements/'
log_name = 'EMT_Trafo_Elements'
print(work_dir + log_name + '.csv')
trafo_elements_emt = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')

In [ ]:
plt.figure()
plt.plot(trafo_elements_emt['v1_0'].time, PEAK1PH_TO_RMS3PH*trafo_elements_emt['v1_0'].values, label='v1_0')
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(trafo_elements_emt['itrafo_0'].time, PEAK1PH_TO_RMS3PH*trafo_elements_emt['itrafo_0'].values, label='itrafo_0')
plt.legend()
plt.show()

## EMT Trafo composite model

In [ ]:
work_dir = 'logs/EMT_Trafo_Component/'
log_name = 'EMT_Trafo_Component'
print(work_dir + log_name + '.csv')
trafo_component_emt = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')

In [ ]:
plt.figure()
#plt.plot(trafo_component_emt['v1_0'].time, PEAK1PH_TO_RMS3PH*trafo_component_emt['v1_0'].values, label='v1_0')
#plt.plot(trafo_component_emt['v1_1'].time, PEAK1PH_TO_RMS3PH*trafo_component_emt['v1_1'].values, label='v1_1')
plt.plot(trafo_component_emt['v1_2'].time, PEAK1PH_TO_RMS3PH*trafo_component_emt['v1_2'].values, label='v1_2')
plt.legend()
plt.show()

In [ ]:
plt.figure()
#plt.plot(trafo_component_emt['itrafo_0'].time, PEAK1PH_TO_RMS3PH*trafo_component_emt['itrafo_0'].values, label='itrafo_0')
#plt.plot(trafo_component_emt['itrafo_1'].time, PEAK1PH_TO_RMS3PH*trafo_component_emt['itrafo_1'].values, label='itrafo_1')
plt.plot(trafo_component_emt['itrafo_2'].time, PEAK1PH_TO_RMS3PH*trafo_component_emt['itrafo_2'].values, label='itrafo_2')
plt.legend()
plt.show()

## Error for EMT Trafo

In [ ]:
plt.figure()
#for name in ['v1_0', 'v1_1', 'v1_1', 'itrafo_0', 'itrafo_1', 'itrafo_2']:
for name in ['itrafo_0']:
    plt.plot(trafo_elements_emt[name].time, trafo_elements_emt[name].values - trafo_component_emt[name].values, label=name+'_error')
plt.legend()
plt.show()

## Assertion for EMT Trafo

In [ ]:
errors_emt = []
for name in ['v1_0', 'v1_1', 'v1_1', 'itrafo_0', 'itrafo_1', 'itrafo_2']:
    errors_emt.append(np.absolute(trafo_elements_emt[name].values - trafo_component_emt[name].values).max())
    print(name + ': ' + str(errors_emt[-1]))
assert np.max(errors_emt) < epsilon

### Comparison SP vs. DP

In [ ]:
plt.figure()
for name in [('v1_shift', 'v1_shift')]:
    plt.plot(trafo_component_sp_shifted[name[0]].time, trafo_component_sp_shifted[name[0]].values - trafo_component_dp_shifted[name[1]].values, label=name[0]+' (SP) vs. '+name[1]+' (DP)')
plt.legend()
plt.show()

In [ ]:
plt.figure()
for name in [('itrafo_shift', 'itrafo_shift')]:
    plt.plot(trafo_component_sp_shifted[name[0]].time, trafo_component_sp_shifted[name[0]].values - trafo_component_dp_shifted[name[1]].values, label=name[0]+' (SP) vs. '+name[1]+' (DP)')
plt.legend()
plt.show()

### Assertion SP vs. DP

In [ ]:
compare_errors_abs = []
compare_errors_rel = []
for name in [('v1_shift', 'v1_shift'), ('itrafo_shift', 'itrafo_shift')]:
    compare_errors_abs.append(np.absolute(trafo_component_sp_shifted[name[0]].values - trafo_component_dp_shifted[name[1]].values).max())
    compare_errors_rel.append(np.absolute(trafo_component_sp_shifted[name[0]].values - trafo_component_dp_shifted[name[1]].values).max()/trafo_component_dp_shifted[name[1]].values.max())
    print(name[0]+' vs. '+name[1] + ' (abs): ' + str(compare_errors_abs[-1]))
    print(name[0]+' vs. '+name[1] + ' (rel): ' + str(compare_errors_rel[-1]))
print('Max rel error: '+ '{:.2}'.format(np.max(compare_errors_rel)*100) +'%')
assert np.max(compare_errors_rel) < 1e-12

### Comparison EMT vs. DP

In [ ]:
plt.figure()
for name in [('v1_0', 'v1_shift')]:
    plt.plot(trafo_component_emt[name[0]].time, PEAK1PH_TO_RMS3PH*trafo_component_emt[name[0]].values - trafo_component_dp_shifted[name[1]].values, label=name[0]+' vs. '+name[1])
plt.legend()
plt.show()

In [ ]:
plt.figure()
for name in [('itrafo_0', 'itrafo_shift')]:
    plt.plot(trafo_component_emt[name[0]].time, PEAK1PH_TO_RMS3PH*trafo_component_emt[name[0]].values - trafo_component_dp_shifted[name[1]].values, label=name[0]+' vs. '+name[1])
plt.legend()
plt.show()

### Assertion EMT vs. DP

In [ ]:
compare_errors_abs = []
compare_errors_rel = []
for name in [('v1_0', 'v1_shift'), ('itrafo_0', 'itrafo_shift')]:
    compare_errors_abs.append(np.absolute(PEAK1PH_TO_RMS3PH*trafo_component_emt[name[0]].values - trafo_component_dp_shifted[name[1]].values).max())
    compare_errors_rel.append(np.absolute(PEAK1PH_TO_RMS3PH*trafo_component_emt[name[0]].values - trafo_component_dp_shifted[name[1]].values).max()/trafo_component_dp_shifted[name[1]].values.max())
    print(name[0]+' vs. '+name[1] + ' (abs): ' + str(compare_errors_abs[-1]))
    print(name[0]+' vs. '+name[1] + ' (rel): ' + str(compare_errors_rel[-1]))
print('Max rel error: '+ '{:.2}'.format(np.max(compare_errors_rel)*100) +'%')
assert np.max(compare_errors_rel) < 0.012
#assert np.max(compare_errors_rel) < 1e-4